In [ ]:
%matplotlib inline


神经网络
===============

神经网络可以用``torch.nn``包来构建。

 ``nn``依赖于``autograd`` 定义模型并对其进行区分。
一个 ``nn.Module`` 包含layers,和一个method ``forward(input)``用来返回``output``.

以手写数字识别为例：
<center><img src="img/mnist.png" width="50%"/></center>
这是一个简单的前馈网络。它接受输入通过一些层，最终给出输出。

神经网络的典型训练程序如下：

- 定义具有可学习参数（或权重）的神经网络
- 在一个输入的数据集上迭代
- 通过网络处理输入
- 计算损失（与正确的输出距离有多远）
- 反向传播梯度给网络的参数
- 更新网络的权重，通常使用一个简单的更新规则：
  ``weight = weight - learning_rate * gradient``

定义网络
------------------




In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
        self.conv1 = nn.Conv2d(1, 6, 5) #输入为1通道，输出为6通道，kernel size:5x5
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120) #最后一个池化层后为5x5x16的feature map，因此全连接层尺寸为（16x5x5,120）
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 尺寸是正方形，可以只指定一个数字
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # 取除了batch之外的所有维度，因为二维卷积层输入矩阵的维度为 (N,Cin,H,W)
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


你只需要定义``forward``函数，``backward``函数 (用来计算梯度) 是自动为你定义的。
你可以在``forward``函数中使用任何``Tensor``运算。

模型的可学习参数由``net.parameters()``返回



In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


让我们尝试一个随机的32x32输入
注意：这个网络的期望输入大小（LeNet）是32x32。要使用这个网络在MNIST数据集，请将数据调整到32x32。



In [5]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0155,  0.0552,  0.0103, -0.1088, -0.0398, -0.0659, -0.0163,
          0.0116, -0.1223, -0.0396]])


所有参数的梯度缓冲区和随机梯度置为0



In [6]:
net.zero_grad()
out.backward(torch.randn(1, 10))

<div class="alert alert-info"><h4>Note</h4><p>``torch.nn``只支持mini-batches. 整个``torch.nn``只支持输入为小批量样本,并且不能是单个样本.

    比如, ``nn.Conv2d`` 接受一个4D的Tensor：
    ``nSamples x nChannels x Height x Width``.

   如果只用一个样本,使用``input.unsqueeze(0)`` 添加一个假的batch维度.</p></div>

在继续之前，回顾一下目前看到的所有类。

**回顾:**
  -  ``torch.Tensor`` - 一个*多维数组*支持autograd操作，例如``backward()``。也保存了关于tensor的梯度
  -  ``nn.Module`` - 神经网络模块。 *封装参数的方便方法*, ：帮助转移到GPU，导出，加载等
  -  ``nn.Parameter`` - 一种Tensor, 当把它赋值给一个Module时,被自动的注册为一个参数。
  -  ``autograd.Function`` - 实现一个自动求导操作的前向和反向定义,每个tensor操作至少创建一个函数节点,它连接到产生一个tensor的函数上并对其历史进行编码

**到这里，已经讨论了：**
  -  定义一个神经网络
  -  处理输入和调用backward

**剩下的内容：**
  -  计算损失值
  -  更新神经网络的权值
损失函数
-------------
一个损失函数接受一对`(output, target)`作为输入(output为网络的输出,target为实际值),计算一个值来估计网络的输出和目标值相差多少.

在nn包中有几种不同的损失函数 <http://pytorch.org/docs/nn.html#loss-functions>。
一个简单的损失函数是：``nn.MSELoss``：它计算网络预测和目标之间的均方误差。

例如：



In [7]:
output = net(input)
target = torch.arange(1, 11)  # 一个虚拟目标
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(38.9883)


现在,你反向跟踪`loss`,使用它的`.grad_fn`属性,你会看到向下面这样的一个计算图:

    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> view -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss

所以, 当你调用 ``loss.backward()``, 整个图关于损失被求导，图中所有的Tensor的``requres_grad=True``
所以``.grad`` Tensor累计它们的梯度。

为了说明,我们反向跟踪几步:


In [8]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

反向传播
--------
为了反向传播误差,我们所需做的是调用``loss.backward()``。
你需要清除已存在的梯度,否则梯度将被累加到已存在的梯度上。

现在，将调用``loss.backward()``，并查看conv1层的偏置项在反向传播前后的梯度。



In [9]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([ 0.,  0.,  0.,  0.,  0.,  0.])
conv1.bias.grad after backward
tensor([ 0.1015,  0.0653, -0.0098,  0.0378, -0.0393, -0.0104])


现在，我们已经看到了如何使用损失函数。

**稍后阅读：**

 神经网络包包含了各种用来构成深度神经网络的模块和损失函数,一份完整的文档查看[这里](http://pytorch.org/docs/nn>)。

更新权重
------------------
实践中最简单的更新规则是随机梯度下降(SGD):

     ``weight = weight - learning_rate * gradient``

我们可以使用简单的Python代码实现这个规则：

```python

    learning_rate = 0.01
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)
```
然而，当你使用神经网络是，你想要使用各种不同的更新规则，比如SGD，Nesterov-SGD，Adam，MSPROP等。为了做到这一点，构建了一个包``torch.optim``实现了所有的这些规则。使用他们非常简单：



In [10]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update